<a href="https://colab.research.google.com/github/Gabrielghj/Bot-de-Trading-con-Backtrader/blob/main/Bot_de_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importación de librerías

In [ ]:
import backtrader as bt
import pandas as pd

# Backtrader: Librería para backtesting (simulación de estrategias).
# Pandas: Se usa para leer y manipular los datos históricos de BTC.

Cargar datos históricos

In [ ]:
data = pd.read_csv('btc_data.csv', index_col='Date', parse_dates=True)

# Cargamos los datos desde un archivo CSV llamado "btc_data.csv".
# La columna "Date" se usa como índice con formato de fecha.

Crear un Data Feed para Backtrader

In [ ]:
data_feed = bt.feeds.PandasData(dataname=data)

# Convertimos los datos de Pandas a un formato que Backtrader pueda manejar (Data Feed).

Crear el "Cerebro" de Backtrader

In [ ]:
cerebro = bt.Cerebro()
cerebro.adddata(data_feed)

# Backtrader usa un objeto llamado Cerebro como el motor que maneja la estrategia, las operaciones y los gráficos.
# adddata(data_feed) carga los datos históricos en el cerebro.


Configurar el capital inicial

In [ ]:
cerebro.broker.setcash(10000.0)

# Se establece un capital inicial de $10,000 para la simulación.

Configurar el tamaño de la posición

In [ ]:
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

# Se define que cada operación va a comprar o vender 1 unidad de BTC.

Configurar la comisión por operación

In [ ]:
cerebro.broker.setcommission(commission=0.001)

# Se aplica una comisión del 0.1% por operación (típica en exchanges como Binance o Coinbase).


Calcular el RSI

In [ ]:
rsi_period = 14
rsi = bt.indicators.RSI_SMA(data_feed.close, period=rsi_period)

# Calculamos el RSI (Relative Strength Index) con un período de 14 días.
# RSI indica si el mercado está sobrecomprado (>70) o sobrevendido (<30).

Iniciar variables para rastrear la posición

In [ ]:
position_open = False

# position_open almacena si actualmente tenemos BTC o no.
# True → Tenemos BTC.
# False → No tenemos BTC.

 Aplicar la estrategia de compra y venta

In [ ]:
for i in range(len(data)):
    current_rsi = rsi[0]
    current_price = data['Close'].iloc[i]

# Se recorre cada fila del dataset (cada día o periodo).
# Se obtiene el valor actual del RSI y el precio de cierre.

Condición de compra

In [ ]:
if not position_open and current_rsi < 30:
    cerebro.broker.buy(price=current_price)
    position_open = True
    print(f"Comprado a {current_price} en la fecha {data.index[i]}")

# Si no tenemos BTC y el RSI está por debajo de 30, compramos BTC.
# Se marca position_open = True para indicar que tenemos BTC.

Condición de venta

In [ ]:
elif position_open and current_rsi > 70:
    cerebro.broker.sell(price=current_price)
    position_open = False
    print(f"Vendido a {current_price} en la fecha {data.index[i]}")

# Si ya tenemos BTC y el RSI supera 70, vendemos BTC.
# Se marca position_open = False porque ya no tenemos BTC.

Mostrar el valor final del portafolio

In [ ]:
print(f'Valor final del portafolio: {cerebro.broker.getvalue()}')

# Imprime el saldo final después de todas las operaciones.

Graficar los resultados

In [ ]:
cerebro.plot()

# Genera un gráfico con los precios y operaciones realizadas.